# 01 · Ingest from ada-panda-mini

In [14]:
from pathlib import Path
import pandas as pd, yaml

# Use repo root, not parents[1]
CFG = yaml.safe_load(open(str(Path.cwd().parents[0] / "config.yml")))
SRC = Path(CFG["upstream_reports_path"])
assert SRC.exists(), f"Upstream path not found: {SRC}"

tlgs_pq = SRC / "tlgs.parquet"
bench_pq = SRC / "benchmarks.parquet"

print("Looking for:", tlgs_pq)
print("Exists:", tlgs_pq.exists())

Looking for: /Users/cmontefusco/Coding projects/ada-panda-mini/reports/tlgs.parquet
Exists: True


In [16]:
import pandas as pd

tlgs = pd.read_parquet(tlgs_pq)
print("TLGs shape:", tlgs.shape)
print("TLGs columns:", list(tlgs.columns))

needed = {"id","visit_week","ADA_pos","ADA_std_detected","ADA_pan_detected","conc_reported_ngmL"}
missing = needed - set(tlgs.columns)
if missing:
    raise RuntimeError(f"tlgs.parquet missing required columns: {sorted(missing)}")

TLGs shape: (6360, 8)
TLGs columns: ['id', 'time_days', 'visit_week', 'conc_ngmL_true', 'conc_reported_ngmL', 'ADA_pos', 'ADA_std_detected', 'ADA_pan_detected']


In [18]:
from pathlib import Path
import pandas as pd, yaml

CFG = yaml.safe_load(open(str(Path.cwd().parents[0] / "config.yml")))
SRC = Path(CFG["upstream_reports_path"])
assert SRC.exists(), f"Upstream path not found: {SRC}"

tlgs = pd.read_parquet(SRC / "tlgs.parquet") if (SRC / "tlgs.parquet").exists() else pd.DataFrame()
bench = pd.read_parquet(SRC / "benchmarks.parquet") if (SRC / "benchmarks.parquet").exists() else pd.DataFrame()

print("TLGs:", tlgs.shape, "| Benchmarks:", bench.shape)

cols = [c for c in ["id","visit_week","ADA_pos","ADA_std_detected","ADA_pan_detected","conc_reported_ngmL"] if c in tlgs.columns]
ada = tlgs[cols].copy() if cols else pd.DataFrame()
if not ada.empty:
    ada.rename(columns={"id":"USUBJID","visit_week":"VISIT","conc_reported_ngmL":"CONC_NGM"}, inplace=True)
    long = ada.melt(id_vars=["USUBJID","VISIT","CONC_NGM","ADA_pos"],
                    value_vars=[c for c in ["ADA_std_detected","ADA_pan_detected"] if c in ada.columns],
                    var_name="METHOD_FLAG", value_name="ADA_DETECTED")
    long["METHOD"] = long["METHOD_FLAG"].map({"ADA_std_detected":"Standard","ADA_pan_detected":"PandA"})
    long.drop(columns=["METHOD_FLAG"], inplace=True)
else:
    long = pd.DataFrame()

out = Path.cwd().parents[0] / "intermediate" / "ada_harmonized.csv"
out.parent.mkdir(parents=True, exist_ok=True)
long.to_csv(out, index=False)
print("Wrote:", out, "| shape:", long.shape)

TLGs: (6360, 8) | Benchmarks: (4, 5)
Wrote: /Users/cmontefusco/Coding projects/dmpk-standards-interoperability/intermediate/ada_harmonized.csv | shape: (12720, 6)
